In [ ]:
import tensorflow as tf
import zipfile
import os
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_all_data.zip
zipref=zipfile.ZipFile("10_food_classes_all_data.zip")
zipref.extractall()
zipref.close()

--2024-03-25 14:18:45--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_all_data.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.175.207, 74.125.24.207, 142.251.10.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.175.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 519183241 (495M) [application/zip]
Saving to: ‘10_food_classes_all_data.zip’

10_food_classes_all 100%[===================>] 495.13M  22.3MB/s    in 23s     

2024-03-25 14:19:09 (21.5 MB/s) - ‘10_food_classes_all_data.zip’ saved [519183241/519183241]



In all the below models rescaling of image has been done internally hence we dont rescale it while generating images.
In model 1 and 3 data augmentation was used as it helped the model train better and increase accuracy whereas in model 2 data augmentation wasnt used as the changed data made it harder for pre trained model to learn and led to overfitting and accuracy falling. The effectiveness of data augmentation may vary depending on how well the pre-trained model has learned to generalize to new data.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


tf.random.set_seed(42)

# Define ImageDataGenerators for training and testing

train_datagen_augmented = ImageDataGenerator(
    rotation_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    height_shift_range=0.2,
    width_shift_range=0.2

)

test_datagen = ImageDataGenerator(
    #rescale=1./255

)
train_datagen = ImageDataGenerator(
    #rescale=1./255

)

# Flow training images in batches using the generator
train_data_augmented = train_datagen_augmented.flow_from_directory(
    '10_food_classes_all_data/train/',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    seed=42,
    shuffle=True
)

# Flow validation images in batches using the generator
test_data = test_datagen.flow_from_directory(
    '10_food_classes_all_data/test/',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    seed=42
)
train_data=train_datagen.flow_from_directory(
    '10_food_classes_all_data/train/',
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    seed=42,
    shuffle=True
)

Found 7500 images belonging to 10 classes.
Found 2500 images belonging to 10 classes.
Found 7500 images belonging to 10 classes.


In [ ]:
base_model=tf.keras.applications.EfficientNetB4(include_top=False)
base_model.trainable=False #we are taking pre trained model so we shouldnt train it and disturb its accuracy

inputs=tf.keras.layers.Input(shape=(128,128,3))
x=base_model(inputs)
x=tf.keras.layers.GlobalAveragePooling2D()(x)
outputs=tf.keras.layers.Dense(10,activation='softmax')(x)
model=tf.keras.models.Model(inputs,outputs)
model.summary()
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
               optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
               metrics=['accuracy'])

71686520/71686520 [==============================] - 4s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 efficientnetb4 (Functional  (None, None, None, 1792   17673823  
 )                           )                                   
                                                                 
 global_average_pooling2d (  (None, 1792)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 10)                17930     
                                                                 
Total params: 17691753 (67.49 MB)
Trainable params: 17930 (70.04 KB)
Non-trainable params: 17673823 (67.42 MB)
_________________

Why EfficientNet-Scalable: Adaptable across different model sizes, catering to diverse hardware constraints and application requirements.
Transfer Learning: Pre-trained on large datasets facilitate effective transfer learning for various classification tasks.
Generalization: Learns rich hierarchical representations, enabling robust performance across diverse datasets and visual patterns.

In [ ]:
base_model.summary()

Model: "efficientnetb4"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, None, None, 3)]      0         []                            
                                                                                                  
 rescaling (Rescaling)       (None, None, None, 3)        0         ['input_1[0][0]']             
                                                                                                  
 normalization (Normalizati  (None, None, None, 3)        7         ['rescaling[0][0]']           
 on)                                                                                              
                                                                                                  
 rescaling_1 (Rescaling)     (None, None, None, 3)        0         ['normalization[0

In [ ]:
model.fit(train_data_augmented,epochs=3,validation_data=test_data,steps_per_epoch=len(train_data),validation_steps=int(0.2*len(test_data)))

Epoch 1/3
235/235 [==============================] - 76s 254ms/step - loss: 1.2686 - accuracy: 0.6021 - val_loss: 0.7105 - val_accuracy: 0.7812
Epoch 2/3
235/235 [==============================] - 52s 223ms/step - loss: 0.9369 - accuracy: 0.7049 - val_loss: 0.6976 - val_accuracy: 0.7958
Epoch 3/3
235/235 [==============================] - 52s 222ms/step - loss: 0.8737 - accuracy: 0.7116 - val_loss: 0.6469 - val_accuracy: 0.8125


In [ ]:
model.evaluate(test_data)

79/79 [==============================] - 8s 101ms/step - loss: 0.6288 - accuracy: 0.8012


[0.6288015246391296, 0.8011999726295471]

In [ ]:
base_model.trainable=True
for i,layer in enumerate(base_model.layers[:-10]):
  layer.trainable=False

The early layers of a pre-trained model capture generic features such as edges, textures, and basic shapes, which are useful across a wide range of tasks. By freezing these layers and only fine-tuning the last few layers, we allow the model to adapt its higher-level features to the specific characteristics of the new dataset and task. These higher-level features tend to be more task-specific and may require adjustments during fine-tuning to improve classification accuracy.

In [ ]:
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
               optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),#new learning rate after changing layers=old learning rate/10
               metrics=['accuracy'])
model.fit(train_data_augmented,epochs=6,validation_data=test_data,steps_per_epoch=len(train_data),validation_steps=int(0.2*len(test_data)),initial_epoch=3)

Epoch 4/6
235/235 [==============================] - 73s 240ms/step - loss: 0.8732 - accuracy: 0.7143 - val_loss: 0.5439 - val_accuracy: 0.8313
Epoch 5/6
235/235 [==============================] - 53s 225ms/step - loss: 0.7757 - accuracy: 0.7472 - val_loss: 0.4966 - val_accuracy: 0.8542
Epoch 6/6
235/235 [==============================] - 52s 221ms/step - loss: 0.7063 - accuracy: 0.7707 - val_loss: 0.5623 - val_accuracy: 0.8250


In [ ]:
model.evaluate(test_data)

79/79 [==============================] - 7s 93ms/step - loss: 0.5321 - accuracy: 0.8212


[0.5320504903793335, 0.8212000131607056]

MODEL 2-

In [ ]:
base_model_2=tf.keras.applications.Xception(include_top=False)
base_model_2.trainable=False

inputs=tf.keras.layers.Input(shape=(128,128,3))
x=base_model_2(inputs)
x=tf.keras.layers.GlobalAveragePooling2D()(x)
outputs=tf.keras.layers.Dense(10,activation='softmax')(x)
model2=tf.keras.models.Model(inputs,outputs)
model2.summary()
model2.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
               optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
               metrics=['accuracy'])

83683744/83683744 [==============================] - 5s 0us/step
Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 128, 128, 3)]     0         
                                                                 
 xception (Functional)       (None, None, None, 2048   20861480  
                             )                                   
                                                                 
 global_average_pooling2d_4  (None, 2048)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_4 (Dense)             (None, 10)                20490     
                                                                 
Total params: 20881970 (79.66 MB)
Trainable params: 20490 (80.04 KB)
Non-trainable params: 20861480 (79.58 MB)
_______________

Why Xception-Scalable: Adaptable across different model sizes, catering to diverse hardware constraints and application requirements.
Transfer Learning: Pre-trained on large datasets like ImageNet, facilitating effective transfer learning for various classification tasks.
Generalization: Learns rich hierarchical representations, enabling robust performance across diverse datasets and visual patterns.

In [ ]:
base_model_2.summary()

Model: "xception"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_9 (InputLayer)        [(None, None, None, 3)]      0         []                            
                                                                                                  
 block1_conv1 (Conv2D)       (None, None, None, 32)       864       ['input_9[0][0]']             
                                                                                                  
 block1_conv1_bn (BatchNorm  (None, None, None, 32)       128       ['block1_conv1[0][0]']        
 alization)                                                                                       
                                                                                                  
 block1_conv1_act (Activati  (None, None, None, 32)       0         ['block1_conv1_bn[0][0]

In [ ]:
model2.fit(train_data,epochs=3,validation_data=test_data,steps_per_epoch=len(train_data),validation_steps=int(0.2*len(test_data)))

Epoch 1/3
235/235 [==============================] - 27s 95ms/step - loss: 4.8730 - accuracy: 0.1864 - val_loss: 3.4259 - val_accuracy: 0.2333
Epoch 2/3
235/235 [==============================] - 20s 84ms/step - loss: 3.2829 - accuracy: 0.2513 - val_loss: 3.3245 - val_accuracy: 0.2625
Epoch 3/3
235/235 [==============================] - 21s 87ms/step - loss: 2.8300 - accuracy: 0.2880 - val_loss: 3.2142 - val_accuracy: 0.2479


In [ ]:
model2.evaluate(test_data)

79/79 [==============================] - 7s 93ms/step - loss: 3.2205 - accuracy: 0.2452


[3.220451593399048, 0.2451999932527542]

In [ ]:
base_model_2.trainable=True
for i,layer in enumerate(base_model.layers[:-10]):
  layer.trainable=False

In [ ]:
model2.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
               optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
               metrics=['accuracy'])
model2.fit(train_data_augmented,epochs=6,validation_data=test_data,steps_per_epoch=len(train_data),validation_steps=int(0.2*len(test_data)),initial_epoch=3)

Epoch 4/6
235/235 [==============================] - 88s 265ms/step - loss: 1.3393 - accuracy: 0.5673 - val_loss: 0.6521 - val_accuracy: 0.7937
Epoch 5/6
235/235 [==============================] - 60s 255ms/step - loss: 0.6965 - accuracy: 0.7835 - val_loss: 0.4488 - val_accuracy: 0.8646
Epoch 6/6
235/235 [==============================] - 61s 257ms/step - loss: 0.5095 - accuracy: 0.8365 - val_loss: 0.4254 - val_accuracy: 0.8625


In [ ]:
model2.evaluate(test_data)

79/79 [==============================] - 6s 74ms/step - loss: 0.4207 - accuracy: 0.8640


[0.4207106828689575, 0.8640000224113464]

MODEL 3-

In [ ]:
base_model_3=tf.keras.applications.DenseNet169(include_top=False)
base_model_3.trainable=False

inputs=tf.keras.layers.Input(shape=(128,128,3))
x=base_model_3(inputs)
x=tf.keras.layers.GlobalAveragePooling2D()(x)
outputs=tf.keras.layers.Dense(10,activation='softmax')(x)
model3=tf.keras.models.Model(inputs,outputs)
model3.summary()
model3.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
               optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
               metrics=['accuracy'])

51877672/51877672 [==============================] - 3s 0us/step
Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 densenet169 (Functional)    (None, None, None, 1664   12642880  
                             )                                   
                                                                 
 global_average_pooling2d_2  (None, 1664)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_2 (Dense)             (None, 10)                16650     
                                                                 
Total params: 12659530 (48.29 MB)
Trainable params: 16650 (65.04 KB)
Non-trainable params: 12642880 (48.23 MB)
_______________

Why DenseNet-Dense Connectivity: Facilitates feature reuse and propagation throughout the network, promoting stronger feature representations and deeper networks.
Parameter Efficiency: Maximizes parameter efficiency by densely connecting each layer to every other layer, leading to improved gradient flow and feature propagation.
Hierarchical Features: Learns rich hierarchical features at multiple scales, enabling robust representation learning and classification performance.
Reduced Vanishing Gradient: Mitigates vanishing gradient issues by enhancing information flow between layers, facilitating more effective training.

In [ ]:
base_model_3.summary()

Model: "densenet169"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_5 (InputLayer)        [(None, None, None, 3)]      0         []                            
                                                                                                  
 zero_padding2d (ZeroPaddin  (None, None, None, 3)        0         ['input_5[0][0]']             
 g2D)                                                                                             
                                                                                                  
 conv1/conv (Conv2D)         (None, None, None, 64)       9408      ['zero_padding2d[0][0]']      
                                                                                                  
 conv1/bn (BatchNormalizati  (None, None, None, 64)       256       ['conv1/conv[0][0]']

In [ ]:
model3.fit(train_data_augmented,epochs=3,validation_data=test_data,steps_per_epoch=len(train_data),validation_steps=int(0.2*len(test_data)))

Epoch 1/3
235/235 [==============================] - 70s 242ms/step - loss: 3.7862 - accuracy: 0.2264 - val_loss: 3.6013 - val_accuracy: 0.2604
Epoch 2/3
235/235 [==============================] - 54s 231ms/step - loss: 2.6807 - accuracy: 0.3201 - val_loss: 2.4020 - val_accuracy: 0.3708
Epoch 3/3
235/235 [==============================] - 52s 219ms/step - loss: 2.4182 - accuracy: 0.3593 - val_loss: 2.0624 - val_accuracy: 0.3938


In [ ]:
model3.evaluate(test_data)

79/79 [==============================] - 9s 118ms/step - loss: 2.1634 - accuracy: 0.3944


[2.163377523422241, 0.3944000005722046]

In [ ]:
base_model_3.trainable=True
for i,layer in enumerate(base_model.layers[:-10]):
  layer.trainable=False

In [ ]:
model3.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
               optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
               metrics=['accuracy'])
model3.fit(train_data_augmented,epochs=6,validation_data=test_data,steps_per_epoch=len(train_data),validation_steps=int(0.2*len(test_data)),initial_epoch=3)

Epoch 4/6
235/235 [==============================] - 162s 300ms/step - loss: 0.9512 - accuracy: 0.6932 - val_loss: 0.5445 - val_accuracy: 0.8229
Epoch 5/6
235/235 [==============================] - 64s 271ms/step - loss: 0.4763 - accuracy: 0.8456 - val_loss: 0.5665 - val_accuracy: 0.8167
Epoch 6/6
235/235 [==============================] - 64s 272ms/step - loss: 0.3338 - accuracy: 0.8929 - val_loss: 0.3526 - val_accuracy: 0.8896


In [ ]:
model3.evaluate(test_data)

79/79 [==============================] - 6s 79ms/step - loss: 0.4031 - accuracy: 0.8724


[0.40308061242103577, 0.8723999857902527]